# DOM features experiment. Coninuation
We will be picking off where we left last time. We'll try to do redo the experiments and see the results, but now we'll try to classify the entire dataset as well, both after having it traind on one website, and all of them.

## Constants

In [1]:
URL_SPLIT_PATH = '../data/final/first/split-url/'
LABEL_SPLIT_PATH = '../data/final/first/split-label/'

EXPERIMENT_RESULTS_CSV = '../data/experimental-results/first-experiments.csv'
URL_SPLIT_CSVS = URL_SPLIT_PATH + '*.csv'
LABEL_SPLIT_CSVS = LABEL_SPLIT_PATH + '*.csv'

In [2]:
%matplotlib inline

# standard library
import itertools
import sys, os
import re
import glob
import logging

from urllib.parse import urlparse

# pandas
import pandas as pd
import dask.dataframe as dd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# skelearn
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# local imports
sys.path.append(os.path.join(os.getcwd(), "../src"))
from utils import get_domain_from_url
from experiments import simple_model_experiment, get_dataset_descr_from_filename, rf_eval

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Now thatwe have the scaffolding for the experiments, we can define functions to run tour five experiments in terms of them, with a given model. We will define all the experiments in term of the `simple_model_experiment` function. For each one we will create the decription.

In [3]:
label_cols = ['detail_description_label', 'detail_image_label', 'detail_price_label',
              'detail_title_label', 'list_image_label', 'list_price_label', 'list_title_label']

For the first experiment, we will only use the the csvs with apges of a website containing a label, for both test and train.

In [4]:
# describe the experiments
train_dataset_files = glob.glob(LABEL_SPLIT_CSVS)
train_datasets = [(file, ) + get_dataset_descr_from_filename(file) for file in train_dataset_files]
train_file_df = pd.DataFrame(data=train_datasets, columns=('file', 'website', 'label'))

# merge it with self 
experiments_df = train_file_df.merge(train_file_df, left_index=True, right_index=True, suffixes=('_train', '_test'))
experiments_df.head()  # inspect the experiments

,file_train,website_train,label_train,file_test,website_test,label_test
0,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_description_label,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_description_label
1,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_description_label,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_description_label
2,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_image_label,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_image_label
3,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_price_label,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_price_label
4,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_title_label,../data/final/first/split-label/lajumate.ro-de...,lajumate.ro,detail_title_label


In [5]:
first_experiment_df = experiments_df.copy()  # persist it

For the second one, the tesing set will be the entire website.

In [6]:
train_dataset_files = glob.glob(LABEL_SPLIT_CSVS)
test_dataset_files = glob.glob(URL_SPLIT_CSVS)

train_datasets = [(file, ) + get_dataset_descr_from_filename(file) for file in train_dataset_files]
test_datasets = [(file, ) + get_dataset_descr_from_filename(file) for file in test_dataset_files]

# we need to air the label/website files with their website equivalent
train_file_df = pd.DataFrame(data=train_datasets, columns=('file', 'website', 'label'))
test_file_df = pd.DataFrame(data=test_datasets, columns=('file', 'website', 'label'))

# join them on the same website, with the proper suffixes
experiments_df = train_file_df.merge(test_file_df, on='website',  suffixes=('_train', '_test'))
experiments_df['website_train'] = experiments_df['website_test'] = experiments_df['website']
experiments_df.head()  # inspect the experiments

,file_train,website,label_train,file_test,label_test,website_train,website_test
0,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_description_label,../data/final/first/split-url/www.emag.ro.csv,all,www.emag.ro,www.emag.ro
1,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_image_label,../data/final/first/split-url/www.emag.ro.csv,all,www.emag.ro,www.emag.ro
2,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_price_label,../data/final/first/split-url/www.emag.ro.csv,all,www.emag.ro,www.emag.ro
3,../data/final/first/split-label/www.emag.ro-de...,www.emag.ro,detail_title_label,../data/final/first/split-url/www.emag.ro.csv,all,www.emag.ro,www.emag.ro
4,../data/final/first/split-label/www.emag.ro-li...,www.emag.ro,list_image_label,../data/final/first/split-url/www.emag.ro.csv,all,www.emag.ro,www.emag.ro


In [7]:
second_experiment_df = experiments_df.copy()

For the third one, the train and test are both on the entire website.

In [8]:
# describe the experiments
train_dataset_files = glob.glob(URL_SPLIT_CSVS)
train_datasets = [(file, ) + get_dataset_descr_from_filename(file) for file in train_dataset_files]
train_file_df = pd.DataFrame(data=train_datasets, columns=('file', 'website', 'label'))

# merge it with self 
experiments_df = train_file_df.merge(train_file_df, left_index=True, right_index=True, suffixes=('_train', '_test'))
experiments_df.head()  # inspect the experiments

,file_train,website_train,label_train,file_test,website_test,label_test
0,../data/final/first/split-url/lajumate.ro.csv,lajumate.ro,all,../data/final/first/split-url/lajumate.ro.csv,lajumate.ro,all
1,../data/final/first/split-url/www.aliexpress.c...,www.aliexpress.com,all,../data/final/first/split-url/www.aliexpress.c...,www.aliexpress.com,all
2,../data/final/first/split-url/www.amazon.com.csv,www.amazon.com,all,../data/final/first/split-url/www.amazon.com.csv,www.amazon.com,all
3,../data/final/first/split-url/www.emag.ro.csv,www.emag.ro,all,../data/final/first/split-url/www.emag.ro.csv,www.emag.ro,all
4,../data/final/first/split-url/www.okazii.ro.csv,www.okazii.ro,all,../data/final/first/split-url/www.okazii.ro.csv,www.okazii.ro,all


In [9]:
third_experiment_df = experiments_df.copy()

The fourth one is trained on an entire website and tested on all of them

In [10]:
train_dataset_files = glob.glob(URL_SPLIT_CSVS)
train_datasets = [(file, ) + get_dataset_descr_from_filename(file) for file in train_dataset_files]
train_file_df = pd.DataFrame(data=train_datasets, columns=('file_train', 'website_train', 'label_train'))

experiments_df = train_file_df
experiments_df['file_test'] = URL_SPLIT_CSVS
experiments_df['website_test'] = experiments_df['label_test'] = 'all'
               
experiments_df.head()

,file_train,website_train,label_train,file_test,website_test,label_test
0,../data/final/first/split-url/lajumate.ro.csv,lajumate.ro,all,../data/final/first/split-url/*.csv,all,all
1,../data/final/first/split-url/www.aliexpress.c...,www.aliexpress.com,all,../data/final/first/split-url/*.csv,all,all
2,../data/final/first/split-url/www.amazon.com.csv,www.amazon.com,all,../data/final/first/split-url/*.csv,all,all
3,../data/final/first/split-url/www.emag.ro.csv,www.emag.ro,all,../data/final/first/split-url/*.csv,all,all
4,../data/final/first/split-url/www.okazii.ro.csv,www.okazii.ro,all,../data/final/first/split-url/*.csv,all,all


In [11]:
fourth_experiment_df = experiments_df.copy()

Finally, the last one is trained on all the websites and tested on them as well.

In [12]:
fifth_experiment_df = pd.DataFrame(data={'file_train': URL_SPLIT_CSVS, 
                                         'file_test': URL_SPLIT_CSVS,
                                         'website_train': 'all', 'website_test': 'all', 
                                         'label_train': 'all', 'label_test': 'all'}, index=[0])
fifth_experiment_df

,file_test,file_train,label_test,label_train,website_test,website_train
0,../data/final/first/split-url/*.csv,../data/final/first/split-url/*.csv,all,all,all,all


## Experiment running
Now that we have the datasets to run the experiments on, we can finally run the experiment on each of them respectively. We will save the results in acouple a dataframes which we will persist into a csv.

### Train/test on website subset

In [13]:
first_results_df = simple_model_experiment(map(lambda x: x[1], first_experiment_df.iterrows()), model_func=rf_eval, 
                                           experiment_name='first-random-forest', label_cols=label_cols)

/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-label/www.emag.ro-detail_description_label-*.csv - ../data/final/first/split-label/www.emag.ro-detail_description_label-*.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-label/lajumate.ro-detail_description_label-*.csv - ../data/final/first/split-label/lajumate.ro-detail_description_label-*.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


...

../data/final/first/split-label/www.olx.ro-list_image_label-*.csv - ../data/final/first/split-label/www.olx.ro-list_image_label-*.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-label/www.olx.ro-list_price_label-*.csv - ../data/final/first/split-label/www.olx.ro-list_price_label-*.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-label/www.olx.ro-list_title_label-*.csv - ../data/final/first/split-label/www.olx.ro-list_title_label-*.csv


### Train on website subset. Test on whole website

In [14]:
second_results_df = simple_model_experiment(map(lambda x: x[1], second_experiment_df.iterrows()), model_func=rf_eval, 
                                            experiment_name='second-random-forest', label_cols=label_cols)

/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


../data/final/first/split-label/www.emag.ro-detail_description_label-*.csv - ../data/final/first/split-url/www.emag.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


../data/final/first/split-label/www.emag.ro-detail_image_label-*.csv - ../data/final/first/split-url/www.emag.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

../data/final/first/split-label/www.olx.ro-list_image_label-*.csv - ../data/final/first/split-url/www.olx.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


../data/final/first/split-label/www.olx.ro-list_price_label-*.csv - ../data/final/first/split-url/www.olx.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


../data/final/first/split-label/www.olx.ro-list_title_label-*.csv - ../data/final/first/split-url/www.olx.ro.csv


### Train/test on whole website

In [ ]:
third_results_df = simple_model_experiment(map(lambda x: x[1], third_experiment_df.iterrows()), model_func=rf_eval, 
                                           experiment_name='third-random-forest', label_cols=label_cols)

/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-url/lajumate.ro.csv - ../data/final/first/split-url/lajumate.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-url/www.aliexpress.com.csv - ../data/final/first/split-url/www.aliexpress.com.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


../data/final/first/split-url/www.amazon.com.csv - ../data/final/first/split-url/www.amazon.com.csv
../data/final/first/split-url/www.emag.ro.csv - ../data/final/first/split-url/www.emag.ro.csv
../data/final/first/split-url/www.okazii.ro.csv - ../data/final/first/split-url/www.okazii.ro.csv
../data/final/first/split-url/www.olx.ro.csv - ../data/final/first/split-url/www.olx.ro.csv
../data/final/first/split-url/www.piata-az.ro.csv - ../data/final/first/split-url/www.piata-az.ro.csv


/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nikitautiu/envs/rist/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Train on single website. Test on all

In [ ]:
fourth_results_df = simple_model_experiment(map(lambda x: x[1], fourth_experiment_df.iterrows()), model_func=rf_eval, 
                                            experiment_name='fourth-random-forest', label_cols=label_cols)

### Train/test on all

In [ ]:
fifth_results_df = simple_model_experiment(map(lambda x: x[1], fifth_experiment_df.iterrows()), model_func=rf_eval, 
                                            experiment_name='fifth-random-forest', label_cols=label_cols)

## Saving the results
Ths notebook is a little too crowded to do any proper analysis here. Memory is also pretty low as the experiments are fairly expensive. In order to mitigate the problem, we will save the experiment results and analyze them in a different notebook.

In [ ]:
expermients = [first_results_df, second_results_df, third_results_df, fourth_results_df, fifth_results_df]
pd.concat(expermients, ignore_index=True).to_csv(EXPERIMENT_RESULTS_CSV)